In [157]:
import pandas as pd
import matplotlib.pyplot as plt
import talib
import numpy as np
from sklearn.metrics import classification_report

In [177]:
BOVA11 = pd.read_csv('PETR4.IND.csv')
BOVA11.fillna(BOVA11.mean(axis=0), inplace=True)

In [178]:
BOVA11.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
395,2020-06-23,21.219999,22.070000,21.139999,21.650000,21.649570,78950900.0
396,2020-06-24,21.490000,21.559999,20.709999,21.000000,20.999582,65162400.0
397,2020-06-25,20.910000,21.469999,20.730000,21.469999,21.469574,47263500.0
398,2020-06-26,21.209999,21.469999,20.780001,20.840000,20.839586,55499300.0
399,2020-06-29,21.100000,21.660000,20.930000,21.660000,21.659571,59809600.0


In [179]:
Close_BOVA11 = BOVA11['Close']
Close_BOVA11 = np.array(Close_BOVA11) 

In [180]:
def normalize(x):
    norm = []
    for i in x:
        z = (i-np.mean(x))/np.std(x)
        norm.append(z)
    return np.array(norm)

In [181]:
# #moving avarage
ma = talib.MA(Close_BOVA11, timeperiod=30, matype=1)

ma_norm = normalize(ma[29:])

In [182]:
ADX = talib.ADX(np.array(BOVA11['High']),np.array(BOVA11['Low']), Close_BOVA11, timeperiod=15)

ADX_norm = normalize(ADX[29:])

In [183]:
macd, macdsignal, macdhist = talib.MACD(Close_BOVA11, fastperiod=12, slowperiod=21, signalperiod=10)
macd_norm = normalize(macd[29:])

In [184]:
MFI = talib.MFI(np.array(BOVA11['High']),np.array(BOVA11['Low']), Close_BOVA11, np.array(BOVA11['Volume']), timeperiod=29)
MFI_norm = normalize(MFI[29:])

In [185]:
def vol_norm(x):
    norm = []
    for i in x:
        z = (i - min(x))/(max(x)-min(x))
        norm.append(z)
    return np.array(norm)

In [186]:
BOVA11_LG = np.log(BOVA11['Close']).diff()

In [187]:
vol = vol_norm(np.array(BOVA11['Volume']))
BOVA_ARRAY = np.array(BOVA11_LG)
Index = BOVA11['Date'].tail(371)
len(Index)

371

In [188]:
def output(y):
    out = []
    for i in y:
        if i < 0:
            out.append(0)
        else:
            out.append(1)
    return np.array(out)

In [189]:
out = output(BOVA_ARRAY)
out.size

400

In [190]:
INPUT_DATA = pd.DataFrame ({'LOG_RETURN':BOVA_ARRAY[29:] , 'VOLUME': vol[29:] , 'MA': ma_norm, 'ADX': ADX_norm, 'MACD':macd_norm, 'MFI': MFI_norm, 'OUTPUT':out[29:] }, index = Index, columns = ['LOG_RETURN', 'VOLUME','MA','ADX','MACD', 'MFI','OUTPUT'])
#LOGDATA.fillna(0, inplace=True)
INPUT_DATA.head()

,LOG_RETURN,VOLUME,MA,ADX,MACD,MFI,OUTPUT
Date,,,,,,,
2018-12-28,0.045555,0.180625,-0.411206,0.586746,-0.864181,-0.515762,1
2019-01-02,0.059067,0.362587,-0.410439,0.374706,-0.643054,-0.528592,1
2019-01-03,0.024226,0.323020,-0.399667,0.260501,-0.406671,-0.555289,1
2019-01-04,0.002836,0.225098,-0.388397,0.169567,-0.211542,-0.585511,1
2019-01-07,0.015654,0.435444,-0.371208,0.202108,-0.020302,-0.084003,1


In [191]:
#Percentage of 0 and ones:
TEST = INPUT_DATA[int(0.7*len(INPUT_DATA)):]
TEST['OUTPUT'].value_counts(normalize=True) * 100

1    50.892857
0    49.107143
Name: OUTPUT, dtype: float64

In [192]:
#INPUT_DATA.to_csv('PETR4_INPUT.csv')

In [193]:
import random 

lin = np.linspace(-1,1,50)
def randome(x):
    if(lin[x]<0):
        return 0
    else:
        return 1

    
y_rand = []
for i in range(len(TEST['OUTPUT'])):
    y_rand.append(randome(random.randrange(0, 50, 1)))

In [194]:
cl = classification_report(np.array(TEST['OUTPUT']), y_rand, output_dict=True)

In [195]:
pd.DataFrame(cl).transpose()

,precision,recall,f1-score,support
0,0.516667,0.563636,0.539130,55.000000
1,0.538462,0.491228,0.513761,57.000000
accuracy,0.526786,0.526786,0.526786,0.526786
macro avg,0.527564,0.527432,0.526446,112.000000
weighted avg,0.527759,0.526786,0.526219,112.000000


In [198]:
um = 0
zero = 0
for i in y_rand:
    if i == 1:
        um = um +1
    else:
        zero = zero +1
um/(um+zero)

0.4642857142857143